# Python TFM Section

## Data Processing

On this section of the TFM it will be done all the preparation needed for the model:

1. Import all the .csv created on the R section to unify them 
2. Set up a unique data frame where we will have all the variables and information required to the regression model 
3. Unstack the structure for making it more suitable to be used on a model


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
#!conda install --yes pathlib 
#$ python -m pip install pathlib

In [3]:
from pathlib import Path

In [4]:
print(Path.cwd())

C:\Users\ES71531200G\Desktop\Data Science\00.TFM


In [5]:
#Defining path to files from the imports folder
%pwd

file1 = "COSTES_E4E_EUROS.csv"
file2 = "LIQUIDACIONES_EUROS.csv"
file3 = "NUCLEAR_WASTES_EUROS.csv"

file5 = "LIQUIDACIONES_MWH.csv"

File_list = [file1, file2, file3]
del(file1,file2,file3)


File_list

['COSTES_E4E_EUROS.csv', 'LIQUIDACIONES_EUROS.csv', 'NUCLEAR_WASTES_EUROS.csv']

In [6]:
%whos

Variable    Type      Data/Info
-------------------------------
File_list   list      n=3
Path        type      <class 'pathlib.Path'>
file5       str       LIQUIDACIONES_MWH.csv
np          module    <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
pd          module    <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
plt         module    <module 'matplotlib.pyplo<...>\\matplotlib\\pyplot.py'>


Importing directly with the read_csv function retrieves an error due to the encoding used by R during the exportation
The parametres encoding and sep solve the probem

In [7]:
inputpath1 = Path.cwd() / 'Outputs' / 'COSTES_E4E_EUROS.csv' 
df1 = pd.read_csv(inputpath1, sep = ';', header = 0 , encoding = "ISO-8859-1")

I create a df with the same columnames and data types but no rows for using it as the initial frame to append everything alltogheter.

In [8]:
dfTotal = df1[0:0]
del(df1)
dfTotal

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_GRUPO_EMPRESARIAL,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,VALOR,ID_UNIDAD


In [9]:
path_list = []
for file in File_list:
    inputpath = Path.cwd() / 'Outputs' / file
    print (inputpath)
    df1 = pd.read_csv(inputpath, sep = ';', header = 0 , encoding = "ISO-8859-1")
    dfTotal = dfTotal.append(df1)

C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\COSTES_E4E_EUROS.csv
C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\LIQUIDACIONES_EUROS.csv
C:\Users\ES71531200G\Desktop\Data Science\00.TFM\Outputs\NUCLEAR_WASTES_EUROS.csv


C:\Users\ES71531200G\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


The process comes back a warning saying that one column is nmissing in at least one section, so we explore the data to see what's happening

In [10]:
dfTotal.sample(20)

,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION
386,ESPAÑA,CANON HID,EMPRESA1,BP,EUROS,UPR1315,"-8,338978e+03",201703
1334,NaN,I. G. Desvíos y Terciaria,EMPRESA2,GN,EUROS,UPR2269,"-15558,07",201702
5383,NaN,G. Desvios,EMPRESA2,GN,EUROS,UPR104,10089,201708
7596,NaN,Terciaria,EMPRESA2,NC,EUROS,UPR295,30326,201710
6861,NaN,Banda,EMPRESA2,EB,EUROS,UPR367,13685910,201709
4473,ESPAÑA,CENTIMO_VERDE,EMPRESA1,LN,EUROS,UPR2341,"-4,845183e+05",201810
9240,NaN,I. G. Desvíos y Terciaria,EMPRESA1,LN,EUROS,UPR2343,"-99310,51",201712
15021,NaN,VCF,EMPRESA3,CI,EUROS,UPR1721,"580508,1",201808
2834,ESPAÑA,OTROS,EMPRESA1,EB,EUROS,UPR2344,"4,663966e+04",201803
1247,NaN,Terciaria,EMPRESA1,EB,EUROS,UPR2103,"368967,1",201702


We obserb that the file "LIQUIDACIONES_EUROS" doesn't have the column ID_AREA_SISTEMA.
To overpass this situations, I proceed to replace all the NaNs with the right values through the selection of the unique tuples
[ID_AREA_SISTEMA - ID_UPR].

In [11]:
df_aux = dfTotal[['ID_AREA_SISTEMA', 'ID_UPR']].dropna().drop_duplicates()
df_aux.sample(10)

,ID_AREA_SISTEMA,ID_UPR
149,ESPAÑA,UPR304
156,ESPAÑA,UPR417
63,ESPAÑA,UPR1862
91,ESPAÑA,UPR2142
8,ESPAÑA,UPR1205
12,ESPAÑA,UPR1206
126,ESPAÑA,UPR2344
75,ESPAÑA,UPR1864
166,ESPAÑA,UPR74
4,ESPAÑA,UPR116


And now I will replace the values using a left join with pandas

In [12]:
df_merged = pd.merge(dfTotal, df_aux, on='ID_UPR', how='left')
df_merged.sample(10)
df_merged.columns

Index(['ID_AREA_SISTEMA_x', 'ID_CONCEPTO_CTRL', 'ID_GRUPO_EMPRESARIAL',
       'ID_TECNOLOGIA', 'ID_UNIDAD', 'ID_UPR', 'VALOR', 'VERSION',
       'ID_AREA_SISTEMA_y'],
      dtype='object')

In [13]:
df_merged = df_merged.rename(columns={'ID_AREA_SISTEMA_y': 'ID_AREA_SISTEMA'})
df_merged = df_merged.drop(columns="ID_AREA_SISTEMA_x")
print(df_merged.columns)
df_merged.sample(10)

Index(['ID_CONCEPTO_CTRL', 'ID_GRUPO_EMPRESARIAL', 'ID_TECNOLOGIA',
       'ID_UNIDAD', 'ID_UPR', 'VALOR', 'VERSION', 'ID_AREA_SISTEMA'],
      dtype='object')


,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
10587,G. Desvios,EMPRESA1,CI,EUROS,UPR1661,"-231624,1",201708,ESPAÑA
422,OTROS,EMPRESA1,CI,EUROS,UPR1860,"-4,992633e+02",201703,ESPAÑA
13130,VCF,EMPRESA1,CI,EUROS,UPR1860,"-34880,29",201711,ESPAÑA
2928,OTROS,EMPRESA1,BP,EUROS,UPR1315,"-7,029561e-01",201804,ESPAÑA
19359,S. Regulacion,EMPRESA1,GN,EUROS,UPR1851,"-125362,7",201807,PORTUGAL
13690,R. Cobertura,EMPRESA2,GN,EUROS,UPR801,-1082251,201711,NaN
5738,D. Medida Contador,EMPRESA1,HN,EUROS,UPR304,"-695186,3",201701,ESPAÑA
6487,M. Diario,EMPRESA1,NC,EUROS,UPR2491,1184122,201702,ESPAÑA
22998,M. Diario,EMPRESA3,HN,EUROS,UPR74,55408010,201811,ESPAÑA
12774,G. Potencia MP,EMPRESA1,HN,EUROS,UPR74,"889583,7",201710,ESPAÑA


In [14]:
df_merged = df_merged[df_merged['ID_GRUPO_EMPRESARIAL'] == 'EMPRESA1']
df_merged.sample(10)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
9398,Ajuste,EMPRESA1,EB,EUROS,UPR2331,"-270000,9",201706,ESPAÑA
2564,OTROS,EMPRESA1,GN,EUROS,UPR2182,"0,000000e+00",201802,ESPAÑA
1985,IMPUESTO ELECT,EMPRESA1,EB,EUROS,UPR2344,"1,593208e+02",201711,ESPAÑA
5960,G. Desvios,EMPRESA1,BX,EUROS,UPR1206,"-6453,48",201702,ESPAÑA
740,IMPUESTO ELECT,EMPRESA1,NC,EUROS,UPR115,"-2,724356e+06",201705,ESPAÑA
22663,VCF,EMPRESA1,EB,EUROS,UPR2103,"12260,84",201811,ESPAÑA
12924,I. R. Cobertura,EMPRESA1,GN,EUROS,UPR160,-2378872,201711,ESPAÑA
2655,CANON_NC_EST,EMPRESA1,NC,EUROS,UPR77,"-1,196984e+06",201802,ESPAÑA
12559,VCF,EMPRESA1,LN,EUROS,UPR2343,"-78737,65",201710,ESPAÑA
4480,ATR,EMPRESA1,LN,EUROS,UPR2342,"-1,823976e+05",201810,ESPAÑA


In [15]:
df_merged[df_merged['ID_AREA_SISTEMA'].isna()].head(5)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
5079,D. Medida Contador,EMPRESA1,NC,EUROS,UPR1198,"-35305,63",201701,NaN
5080,S. Res. Pot. Adicional,EMPRESA1,NC,EUROS,UPR1198,"-192,8",201701,NaN
5239,Ajuste,EMPRESA1,BP,EUROS,UPR1751,"-374890,6",201701,NaN
5240,Bilateral,EMPRESA1,BP,EUROS,UPR1751,"-213155,6",201701,NaN
5241,D. Medida Contador,EMPRESA1,BP,EUROS,UPR1751,"-22453,97",201701,NaN


There are still NaN values, but thanks to our knowledge from the original data, we know that there are ONLY 2 UPRs with ID_AREA_SISTEMA = 'Portugal', what means that every NaN value right now should be equal to ESPAÑA, so we replace now all the NANs

In [16]:
df_merged = df_merged.fillna('ESPAÑA')
df_merged.sample(10)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA
7622,G. Potencia MP,EMPRESA1,CI,EUROS,UPR1661,1578298,201704,ESPAÑA
4127,ATR,EMPRESA1,GN,EUROS,UPR162,"-2,518831e+06",201809,ESPAÑA
17097,Ajuste,EMPRESA1,BP,EUROS,UPR1752,-51771984,201804,ESPAÑA
16777,Res. Pot. Adicional,EMPRESA1,HN,EUROS,UPR304,"296025,3",201803,ESPAÑA
1987,TASA_ARAGON,EMPRESA1,EB,EUROS,UPR2344,"1,268102e+04",201711,ESPAÑA
14629,M. Diario,EMPRESA1,NC,EUROS,UPR116,"888896,8",201801,ESPAÑA
11172,Terciaria,EMPRESA1,BX,EUROS,UPR726,-44625,201708,ESPAÑA
10389,G. Potencia MP,EMPRESA1,HN,EUROS,UPR74,"611370,7",201707,ESPAÑA
1876,OTROS,EMPRESA1,BP,EUROS,UPR1315,"-2,101388e-01",201711,ESPAÑA
708,CENTIMO_VERDE,EMPRESA1,HN,EUROS,UPR304,"-2,595319e+04",201704,ESPAÑA


In [17]:
df_merged[df_merged['ID_AREA_SISTEMA'].isna()].head(5)

,ID_CONCEPTO_CTRL,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_UNIDAD,ID_UPR,VALOR,VERSION,ID_AREA_SISTEMA


And finally, reorder de columns to the same order we are already used to 

In [18]:
df_merged = df_merged[['VERSION','ID_UPR','ID_TECNOLOGIA','ID_GRUPO_EMPRESARIAL','ID_AREA_SISTEMA','ID_CONCEPTO_CTRL','VALOR']]
df_merged.reset_index()
df_merged.sample(5)

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_GRUPO_EMPRESARIAL,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,VALOR
21888,201810,UPR2182,GN,EMPRESA1,ESPAÑA,M. Intradiarios,54687481
21494,201810,UPR160,GN,EMPRESA1,ESPAÑA,R. Cobertura,330516944
2311,201801,UPR1862,CI,EMPRESA1,ESPAÑA,TASAS_MEDIOAMB,"-5,321852e+04"
17839,201805,UPR1862,CI,EMPRESA1,ESPAÑA,M. Intradiarios,3153164
3643,201807,UPR160,GN,EMPRESA1,ESPAÑA,OTROS,"1,164463e+06"


At this point, I will save the current df "df_merged" for the future visualizatin part, this is the structure desired to represent the Integral Margin of the different power plants and so it is for the temporal evolution of every single one of them.

The problem here seemed to be the data types... so first I tried to convert the column value to numeric directly 
with no success...

The error got, suggested that I should convert the data type to floats but the lesson learnt here was that float type in pandas use dots insted of comma for float


In [19]:
df_merged.dtypes

VERSION                  int64
ID_UPR                  object
ID_TECNOLOGIA           object
ID_GRUPO_EMPRESARIAL    object
ID_AREA_SISTEMA         object
ID_CONCEPTO_CTRL        object
VALOR                   object
dtype: object

In [20]:
df_merged['VALOR'].str.replace(',','.').sample(10)

1379      5.681817e+02
941       3.936960e+04
1885     -4.955929e+04
21131         90084.57
10583         -3004716
4195      0.000000e+00
18947        -4835.882
11307         -3073.94
9461         -64883.72
2458     -9.683860e+03
Name: VALOR, dtype: object

In [21]:
#pd.to_numeric(df_merged['VALOR'])
df_merged['VALOR'] = pd.to_numeric((df_merged['VALOR'].str.replace(',','.')),errors='coerce').fillna(0).astype(np.int64)
df_merged.sample(10)

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_GRUPO_EMPRESARIAL,ID_AREA_SISTEMA,ID_CONCEPTO_CTRL,VALOR
3739,201807,UPR2331,EB,EMPRESA1,ESPAÑA,CANON HID,-228977
17036,201804,UPR1661,CI,EMPRESA1,ESPAÑA,G. Potencia MP,6871920
805,201705,UPR1862,CI,EMPRESA1,ESPAÑA,COSTE_COMBUSTIBLE,-2252863
4841,201812,UPR1207,EB,EMPRESA1,ESPAÑA,PEAJE GEN,-252
11498,201709,UPR1861,CI,EMPRESA1,ESPAÑA,Ajuste,-4775971
16441,201803,UPR1864,GN,EMPRESA1,ESPAÑA,S. Res. Pot. Adicional,-1054
20035,201808,UPR1661,CI,EMPRESA1,ESPAÑA,R. Cobertura,104374159
6399,201702,UPR2331,EB,EMPRESA1,ESPAÑA,D. Medida Contador,-36128
21549,201810,UPR1661,CI,EMPRESA1,ESPAÑA,G. Desvios,14355291
14846,201801,UPR1850,GN,EMPRESA1,PORTUGAL,Terciaria,-6550590


In [22]:
df_merged['VALOR'].sum()

175694686343

In [23]:
#df_merged.to_csv?
df_merged.to_csv(Path.cwd() / 'Outputs' / 'INTEGRATED_MARGIN.csv', sep= ';',index=False)

Now I proceed to unstack or pivot the table to get the suitable structure for modeling
During this procedure, I've faced multiple problems so here I brievely describe the process:

1. First attempts ended on multiple errors such as "Length of passed values is 15227, index implies 1" , "index contains duplicate entries,cannot reshape"
2. It seemed clear that in any moment of the dropping unused columns, I created a duplicity on a register so first thing requieres was to do a group by
3. After done, I reseted the index for setting free all the columns
4. I used the pandas fuction "pivot_table" instead the method .pivot due to the hability of the first one to summing all the values generated with duplicities during the process of resizing.
5. Once pivoted, indexes and headers were a problematic segmentation so I dropped it out and create a new header.


In [24]:
df_pivoted = df_merged[['VERSION','ID_UPR','ID_TECNOLOGIA','ID_CONCEPTO_CTRL','VALOR']]
df_pivoted.shape

(15117, 5)

In [25]:
df_pivoted2 =df_pivoted.groupby(['VERSION','ID_UPR','ID_TECNOLOGIA','ID_CONCEPTO_CTRL']).sum()
print(df_pivoted2.shape)
print(df_pivoted2.columns)
df_pivoted2.sample(5)

(15117, 1)
Index(['VALOR'], dtype='object')


,,,,VALOR
VERSION,ID_UPR,ID_TECNOLOGIA,ID_CONCEPTO_CTRL,
201704,UPR1205,EB,D. Medida Contador,-157582
201711,UPR1662,CI,M. Diario,285466934
201702,UPR2331,EB,D. Medida Contador,-36128
201805,UPR1861,CI,Bilateral,28096481
201808,UPR1207,EB,R. Secundaria,-381343


In [26]:
df_pivoted2= df_pivoted2.reset_index()
df_pivoted2.head(5)

,VERSION,ID_UPR,ID_TECNOLOGIA,ID_CONCEPTO_CTRL,VALOR
0,201701,UPR115,NC,Bilateral,81778497
1,201701,UPR115,NC,CANON_NC_EST,-2543929
2,201701,UPR115,NC,COSTE_COMBUSTIBLE,-1875378
3,201701,UPR115,NC,D. Medida Contador,-7853
4,201701,UPR115,NC,IMPUESTO ELECT,-3190970


In [27]:
df_pivoted2.columns

Index(['VERSION', 'ID_UPR', 'ID_TECNOLOGIA', 'ID_CONCEPTO_CTRL', 'VALOR'], dtype='object')

In [28]:
df_pivoted3 = df_pivoted2.pivot_table( 
                          values=['VALOR'], 
                          index=['VERSION', 'ID_UPR', 'ID_TECNOLOGIA'],
                          columns=['ID_CONCEPTO_CTRL'], 
                          aggfunc=np.sum)
df_pivoted3.sample(10)

VALOR                                 \
ID_CONCEPTO_CTRL              A. No Cobrados AMONIACO        ATR     Ajuste   
VERSION ID_UPR  ID_TECNOLOGIA                                                 
201704  UPR1206 BX                       NaN      NaN        NaN   -36001.0   
201810  UPR2331 EB                       NaN      NaN        NaN  -786701.0   
201702  UPR1751 BP                       NaN      NaN        NaN  -997134.0   
201807  UPR116  NC                       NaN      NaN        NaN        NaN   
201712  UPR1207 EB                       NaN      NaN        NaN        NaN   
201709  UPR1850 GN                       NaN      NaN -1184745.0        NaN   
201701  UPR116  NC                       NaN      NaN        NaN        NaN   
201802  UPR1751 BP                       NaN      NaN        NaN -5558819.0   
201805  UPR1863 CI                       NaN      NaN        NaN  -716350.0   
201804  UPR1862 CI                       NaN      NaN        NaN  -460456.0   

                                                                         \
ID_CONCEPTO_CTRL              BONO_SOCIAL_PEGO      Banda     Bilateral   
VERSION ID_UPR  ID_TECNOLOGIA                                             
201704  UPR1206 BX                         NaN        NaN -8.082000e+03   
201810  UPR2331 EB                         NaN    93958.0           NaN   
201702  UPR1751 BP                         NaN        NaN -5.481220e+05   
201807  UPR116  NC                         NaN        NaN  1.453547e+09   
201712  UPR1207 EB                         NaN        NaN           NaN   
201709  UPR1850 GN                   -208453.0  5706428.0           NaN   
201701  UPR116  NC                         NaN        NaN  6.845539e+07   
201802  UPR1751 BP                         NaN        NaN -1.529340e+05   
201805  UPR1863 CI                         NaN  3862802.0  1.940669e+07   
201804  UPR1862 CI                         NaN   542322.0           NaN   

                                                                 ...  \
ID_CONCEPTO_CTRL              CALIZAS CANON HID CANON_CONCESION  ...   
VERSION ID_UPR  ID_TECNOLOGIA                                    ...   
201704  UPR1206 BX                NaN       NaN             NaN  ...   
201810  UPR2331 EB                NaN  -96692.0        -13885.0  ...   
201702  UPR1751 BP                NaN       NaN             NaN  ...   
201807  UPR116  NC                NaN       NaN             NaN  ...   
201712  UPR1207 EB                NaN  -65879.0             NaN  ...   
201709  UPR1850 GN                NaN       NaN             NaN  ...   
201701  UPR116  NC                NaN       NaN             NaN  ...   
201802  UPR1751 BP                NaN       NaN             NaN  ...   
201805  UPR1863 CI                NaN       NaN             NaN  ...   
201804  UPR1862 CI                NaN       NaN             NaN  ...   

                                                                             \
ID_CONCEPTO_CTRL              Redespachos Res. Pot. Adicional S. Regulacion   
VERSION ID_UPR  ID_TECNOLOGIA                                                 
201704  UPR1206 BX                    NaN                 NaN           NaN   
201810  UPR2331 EB                    NaN                 NaN           NaN   
201702  UPR1751 BP               -12190.0                 NaN           NaN   
201807  UPR116  NC                    NaN                 NaN           NaN   
201712  UPR1207 EB                    NaN                 NaN           NaN   
201709  UPR1850 GN                    NaN                 NaN     -118496.0   
201701  UPR116  NC                    NaN                 NaN           NaN   
201802  UPR1751 BP              -131800.0                 NaN           NaN   
201805  UPR1863 CI                    NaN           1718054.0           NaN   
201804  UPR1862 CI                    NaN           1646513.0           NaN   

                                                                    \


In [29]:
df_pivoted3.columns = df_pivoted3.columns.droplevel()
df_modelize= df_pivoted3.reset_index()
df_modelize.head(10)

ID_CONCEPTO_CTRL,VERSION,ID_UPR,ID_TECNOLOGIA,A. No Cobrados,AMONIACO,ATR,Ajuste,BONO_SOCIAL_PEGO,Banda,Bilateral,...,Redespachos,Res. Pot. Adicional,S. Regulacion,S. Res. Pot. Adicional,SERV_GEST_RES,TASAS_MEDIOAMB,TASA_ARAGON,Terciaria,UREA,VCF
0,201701,UPR115,NC,NaN,NaN,NaN,NaN,NaN,NaN,81778497.0,...,NaN,NaN,NaN,-36.0,-5156083.0,NaN,NaN,NaN,NaN,NaN
1,201701,UPR116,NC,NaN,NaN,NaN,NaN,NaN,NaN,68455391.0,...,NaN,NaN,NaN,NaN,-4357224.0,NaN,NaN,NaN,NaN,NaN
2,201701,UPR1198,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-192.0,NaN,NaN,NaN,NaN,NaN,NaN
3,201701,UPR1205,EB,NaN,NaN,NaN,-124366.0,NaN,NaN,1578792.0,...,NaN,NaN,NaN,-455.0,NaN,-33711.0,NaN,55144.0,NaN,NaN
4,201701,UPR1206,BX,NaN,NaN,NaN,-9972.0,NaN,NaN,NaN,...,NaN,NaN,NaN,-7.0,NaN,-16346.0,NaN,NaN,NaN,NaN
5,201701,UPR1207,EB,NaN,NaN,NaN,-198314.0,NaN,15066.0,59653.0,...,NaN,NaN,NaN,-807.0,NaN,-1528.0,NaN,225821.0,NaN,999.0
6,201701,UPR1314,BP,NaN,NaN,NaN,-429377.0,NaN,NaN,-118383.0,...,-8556.0,NaN,NaN,-39.0,NaN,NaN,NaN,-24031.0,NaN,NaN
7,201701,UPR1315,BP,NaN,NaN,NaN,-506708.0,NaN,2001.0,NaN,...,1146.0,NaN,NaN,-397.0,NaN,NaN,NaN,441646.0,NaN,-162.0
8,201701,UPR160,GN,-994654.0,NaN,-938689.0,NaN,NaN,1045907.0,NaN,...,47854.0,51355.0,NaN,-2223.0,NaN,NaN,NaN,251416.0,NaN,-14892.0
9,201701,UPR162,GN,-418200.0,NaN,-507356.0,NaN,NaN,957825.0,NaN,...,745956.0,579221.0,NaN,-316.0,NaN,NaN,NaN,573346.0,NaN,-17674.0


Finally, we incorporate now the last column of data that we are gonna implement to the model, the power column from the second
dataframe that we got in the liquidations R process

In [30]:
df_power = pd.read_csv(Path.cwd() / 'Outputs' / 'LIQUIDACIONES_MWH.csv' , sep = ';', header = 0 , encoding = "ISO-8859-1",decimal=',')
print(df_power.describe())
df_power.sample(5)

             VERSION         VALOR
count   13048.000000  1.304800e+04
mean   201754.106913  4.498754e+05
std        49.951242  2.031215e+06
min    201701.000000 -5.489374e+06
25%    201706.000000 -9.301933e+03
50%    201712.000000  2.225860e+03
75%    201806.000000  8.903905e+04
max    201812.000000  3.518217e+07


,VERSION,ID_UPR,ID_GRUPO_EMPRESARIAL,ID_TECNOLOGIA,ID_CONCEPTO_CTRL,ID_UNIDAD,VALOR
2751,201705,UPR74,EMPRESA3,HN,M. Diario,MWH,150257.50
7585,201802,UPR1862,EMPRESA1,CI,D. Medida Contador,MWH,77405.69
12343,201811,UPR1761,EMPRESA2,BP,G. Desvios,MWH,-76978.10
9165,201805,UPR2133,EMPRESA2,EB,R. Secundaria,MWH,74432.47
10105,201807,UPR1761,EMPRESA2,BP,G. Desvios,MWH,-298709.60


I apply the same filters that I did in the previous dfs

And eventually a group by just in case we have the same problem than before

In [31]:
df_power = df_power[df_power['ID_GRUPO_EMPRESARIAL'] == 'EMPRESA1']
print(df_power.describe())

             VERSION         VALOR
count    6417.000000  6.417000e+03
mean   201755.352189  4.508979e+05
std        50.180305  2.115280e+06
min    201701.000000 -3.789073e+06
25%    201706.000000 -1.387000e+04
50%    201712.000000  2.163250e+02
75%    201807.000000  5.869300e+04
max    201812.000000  3.256008e+07


In [32]:
df_power = df_power[['VERSION','ID_UPR','ID_TECNOLOGIA','VALOR']]
df_power= df_power.groupby(['VERSION','ID_UPR','ID_TECNOLOGIA']).sum().reset_index()
print(df_power.describe())

             VERSION         VALOR
count    1024.000000  1.024000e+03
mean   201755.424805  2.825598e+06
std        50.143803  4.930549e+06
min    201701.000000 -3.789073e+06
25%    201706.000000  2.363667e+04
50%    201712.000000  5.563538e+05
75%    201806.000000  3.699485e+06
max    201812.000000  3.262049e+07


In [33]:
df_power.describe()

,VERSION,VALOR
count,1024.000000,1.024000e+03
mean,201755.424805,2.825598e+06
std,50.143803,4.930549e+06
min,201701.000000,-3.789073e+06
25%,201706.000000,2.363667e+04
50%,201712.000000,5.563538e+05
75%,201806.000000,3.699485e+06
max,201812.000000,3.262049e+07


In [34]:
df_modelize = pd.merge(df_modelize, df_power, on=['VERSION','ID_UPR','ID_TECNOLOGIA'], how='left')
df_modelize = df_modelize.rename(columns={'VALOR': 'POWER_MWH'})
df_modelize.sample(10)

,VERSION,ID_UPR,ID_TECNOLOGIA,A. No Cobrados,AMONIACO,ATR,Ajuste,BONO_SOCIAL_PEGO,Banda,Bilateral,...,Res. Pot. Adicional,S. Regulacion,S. Res. Pot. Adicional,SERV_GEST_RES,TASAS_MEDIOAMB,TASA_ARAGON,Terciaria,UREA,VCF,POWER_MWH
928,201810,UPR1860,CI,NaN,NaN,NaN,-147776.0,NaN,15686734.0,81296276.0,...,2851028.0,NaN,-15095.0,NaN,-49363.0,NaN,2272734.0,-47993.0,-101635.0,8728480.080
385,201709,UPR2491,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,-50637.0,NaN,NaN,NaN,NaN,NaN,123618.071
53,201702,UPR162,GN,NaN,NaN,-219407.0,NaN,NaN,3380503.0,NaN,...,38404.0,NaN,-2.0,NaN,NaN,NaN,311562.0,NaN,-137018.0,389916.426
342,201708,UPR2622,NC,NaN,NaN,NaN,NaN,NaN,NaN,391300278.0,...,NaN,NaN,-141.0,-3722441.0,NaN,NaN,NaN,NaN,NaN,8243796.258
476,201711,UPR304,HN,NaN,NaN,NaN,NaN,NaN,3689228.0,27680178.0,...,NaN,NaN,-7620.0,NaN,-7898.0,NaN,133010.0,NaN,8772.0,3804559.690
739,201805,UPR77,NC,NaN,NaN,NaN,NaN,NaN,NaN,447234006.0,...,NaN,NaN,-1991.0,-1847333.0,-1315660.0,NaN,NaN,NaN,NaN,9035556.870
766,201806,UPR2331,EB,NaN,NaN,NaN,-1498433.0,NaN,552847.0,18182295.0,...,NaN,NaN,-5604.0,NaN,-27475.0,NaN,2652945.0,NaN,-19109.0,795811.152
433,201710,UPR304,HN,NaN,NaN,NaN,-1058067.0,NaN,3583622.0,NaN,...,NaN,NaN,-242325.0,NaN,-10443.0,NaN,-909650.0,NaN,-118518.0,2685409.750
57,201702,UPR1752,BP,NaN,NaN,NaN,-1633622.0,NaN,NaN,NaN,...,NaN,NaN,-91.0,NaN,NaN,NaN,410028.0,NaN,NaN,55424.017
603,201802,UPR300,GN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,-224.0,NaN,0.0,NaN,NaN,NaN,NaN,-9567.918


In [35]:
df_modelize.to_csv(Path.cwd() / 'Outputs' / 'DF_MODELIZE.csv', sep= ';',decimal=',',index=False)

In [36]:
To_be_deleted =['df1',
                'df_aux',
                'df_merged',
                'dfTotal',
                'df_pivoted',
                'df_pivoted2',
                'df_pivoted3',
                'path_list',
                'inputpath',
                'inputpath1']
To_be_deleted

['df1',
 'df_aux',
 'df_merged',
 'dfTotal',
 'df_pivoted',
 'df_pivoted2',
 'df_pivoted3',
 'path_list',
 'inputpath',
 'inputpath1']

In [37]:
for item in To_be_deleted:
    try:
        del item
    except:
        pass